# M2177.003100 Deep Learning <br> Final Proejct: Text to Image Synthesis (Tensorflow)

Copyright (C) Data Science & AI Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. 

**For understanding of this work, please carefully look at given PPT file.**

**Note**: certain details are missing or ambiguous on purpose, in order to test your knowledge on the related materials. However, if you really feel that something essential is missing and cannot proceed to the next step, then contact the teaching staff with clear description of your problem.

### Submitting your work:
<font color=red>**DO NOT clear the training process **</font> so that TAs can grade both your code and results.  
**The TA will set a config file as 'eval_birds.yml' when evaluating the code using 'hidden test dataset'. Thus, please make sure that your code can generate proper data to measure inception score and R-precision of 'hidden test dataset'.**

## 1. Load datasets
The Birds dataset will be downloaded automatically if it is not located in the *data* directory. <br>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os, nltk
from miscc.config import cfg, cfg_from_file
import pprint
import datetime
import dateutil.tz
import numpy as np
import scipy
from utils.data_utils import CUBDataset
from utils.loss import cosine_similarity
import pandas as pd
from scipy.io import loadmat
import re
import string
import random
import time

%matplotlib inline

/home/chszerg/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/chszerg/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/chszerg/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/chszerg

In [2]:
# Set a config file as 'train_birds.yml' in training, as 'eval_birds.yml' for evaluation
cfg_from_file('cfg/train_birds.yml') # eval_birds.yml

print('Using config:')
pprint.pprint(cfg)

os.environ['CUDA_VISIBLE_DEVICES'] = cfg.GPU_ID

now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
output_dir = 'sample/%s_%s_%s' % (cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp)

Using config:
{'BATCH_SIZE': 64,
 'CHECKPOINT_DIR': './checkpoint',
 'CHECKPOINT_NAME': 'model.ckpt',
 'CNN': {'EMBEDDING_DIM': 0, 'H_DIM': 0},
 'CONFIG_NAME': 'text-to-image',
 'CUDA': False,
 'DATASET_NAME': 'birds',
 'DATA_DIR': 'data/birds',
 'EMBEDDING_TYPE': 'cnn-rnn',
 'GAN': {'B_ATTENTION': False,
         'B_CONDITION': False,
         'B_DCGAN': False,
         'CONDITION_DIM': 0,
         'DF_DIM': 0,
         'EMBEDDING_DIM': 0,
         'GF_DIM': 0,
         'R_NUM': 0,
         'Z_DIM': 512},
 'GPU_ID': '0',
 'IMAGE_SIZE': 256,
 'NUM_BATCH_FOR_TEST': 0,
 'RANDOM_SEED': 0,
 'RNN': {'EMBEDDING_DIM': 0,
         'H_DIM': 0,
         'TYPE': '',
         'VOCAB_SIZE': 0,
         'WORD_EMBEDDING_DIM': 0},
 'R_PRECISION_DIR': './evaluation',
 'R_PRECISION_FILE': 'r_precision.npz',
 'R_PRECISION_FILE_HIDDEN': 'r_precision_hidden.npz',
 'TEST': {'B_EXAMPLE': False,
          'GENERATED_HIDDEN_TEST_IMAGES': './evaluation/generated_images_hidden',
          'GENERATED_TEST_IMAGES'

/home/chszerg/final-project-deep-learning-19-tf/miscc/config.py:121: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_cfg = edict(yaml.load(f))


In [3]:
train_dataset = CUBDataset(cfg.DATA_DIR, split='train')
test_dataset = CUBDataset(cfg.DATA_DIR, split='test')

self.current_dir:
/home/chszerg/final-project-deep-learning-19-tf

self.data_dir:
/home/chszerg/final-project-deep-learning-19-tf/data/birds

self.image_dir:
/home/chszerg/final-project-deep-learning-19-tf/data/birds/CUB_200_2011.tgz

Dataset already exists
self.image_dir:
/home/chszerg/final-project-deep-learning-19-tf/data/birds/CUB_200_2011/images

Load from:  data/birds/captions.pickle
self.current_dir:
/home/chszerg/final-project-deep-learning-19-tf

self.data_dir:
/home/chszerg/final-project-deep-learning-19-tf/data/birds

self.image_dir:
/home/chszerg/final-project-deep-learning-19-tf/data/birds/CUB_200_2011.tgz

Dataset already exists
self.image_dir:
/home/chszerg/final-project-deep-learning-19-tf/data/birds/CUB_200_2011/images

Load from:  data/birds/captions.pickle


In [4]:
train_images = train_dataset.images
test_images = test_dataset.images
train_captions = np.asarray(train_dataset.captions_ids)
test_captions = np.asarray(test_dataset.captions_ids)
print(train_images.shape)
print(test_images.shape)
print(train_captions.shape)
print(test_captions.shape)

(8855, 256, 256, 3)
(2933, 256, 256, 3)
(88550, 20)
(29330, 20)


In [5]:
from skimage.transform import resize
train_images_64 = []
for train_image in train_images:
    train_images_64.append(resize(train_image, (64, 64, 3)))
train_images_64 = np.asarray(train_images_64)
print(train_images_64.shape)
assert train_images_64.shape[0] == train_images.shape[0]
test_images_64 = []
for test_image in test_images:
    test_images_64.append(resize(test_image, (64, 64, 3)))
test_images_64 = np.asarray(test_images_64)
print(test_images_64.shape)
assert test_images_64.shape[0] == test_images.shape[0]

(8855, 64, 64, 3)
(2933, 64, 64, 3)


In [6]:
n_captions_train = len(train_captions)
n_captions_per_image = 10
n_images_train = len(train_images)
print(train_images.shape)

(8855, 256, 256, 3)


In [18]:
import scipy.misc
import threading
import scipy.ndimage as ndi
from skimage import transform
from skimage import exposure
import skimage
from nltk.tokenize import RegexpTokenizer

def sent2ID(sample_sentence):
    caption = []
    cap = sample_sentence
    if len(cap) == 0:
        exit()
    cap = cap.replace("\ufffd\ufffd", " ")
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(cap.lower())
    tokens_new = []
    for t in tokens:
        t = t.encode('ascii', 'ignore').decode('ascii')
        if len(t) > 0:
            tokens_new.append(t)
    caption.append(tokens_new)
    caption_new = []
    t = caption[0]
    rev = []
    for w in t:
        if w in train_dataset.wordtoix:
            rev.append(train_dataset.wordtoix[w])
    x, x_len = train_dataset.get_caption(rev)
    caption_new.append(np.squeeze(x, axis=1))
    return caption_new

def ID2sent(sample_caption):
    sentence = []
    for ID in sample_caption:
        if ID != train_dataset.ixtoword['<PAD>']:
            sentence.append(train_dataset.ixtoword[ID])
    return sentence

def get_random_int(min=0, max=10, number=5):
    return [random.randint(min,max) for p in range(0,number)]

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1], 3))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image
    return img

def imsave(images, size, path):
    return scipy.misc.imsave(path, merge(images, size))

def save_images(images, size, image_path):
    return imsave(images, size, image_path)

def threading_data(data=None, fn=None, **kwargs):
    def apply_fn(results, i, data, kwargs):
        results[i] = fn(data, **kwargs)
    results = [None] * len(data)
    threads = []
    for i in range(len(data)):
        t = threading.Thread(
                        name='threading_and_return',
                        target=apply_fn,
                        args=(results, i, data[i], kwargs)
                        )
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
    return np.asarray(results)

def apply_transform(x, transform_matrix, channel_index=2, fill_mode='nearest', cval=0., order=1):
    x = np.rollaxis(x, channel_index, 0)
    final_affine_matrix = transform_matrix[:2, :2]
    final_offset = transform_matrix[:2, 2]
    channel_images = [ndi.interpolation.affine_transform(x_channel, final_affine_matrix,
                      final_offset, order=order, mode=fill_mode, cval=cval) for x_channel in x]
    x = np.stack(channel_images, axis=0)
    x = np.rollaxis(x, 0, channel_index + 1)
    return x

def transform_matrix_offset_center(matrix, x, y):
    o_x = float(x) / 2 + 0.5
    o_y = float(y) / 2 + 0.5
    offset_matrix = np.array([[1, 0, o_x], [0, 1, o_y], [0, 0, 1]])
    reset_matrix = np.array([[1, 0, -o_x], [0, 1, -o_y], [0, 0, 1]])
    transform_matrix = np.dot(np.dot(offset_matrix, matrix), reset_matrix)
    return transform_matrix

def rotation(x, rg=20, is_random=False, row_index=0, col_index=1, channel_index=2,
                    fill_mode='nearest', cval=0.):
    if is_random:
        theta = np.pi / 180 * np.random.uniform(-rg, rg)
    else:
        theta = np.pi / 180 * rg
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
                                [np.sin(theta), np.cos(theta), 0],
                                [0, 0, 1]])
    h, w = x.shape[row_index], x.shape[col_index]
    transform_matrix = transform_matrix_offset_center(rotation_matrix, h, w)
    x = apply_transform(x, transform_matrix, channel_index, fill_mode, cval)
    return x

def crop(x, wrg, hrg, is_random=False, row_index=0, col_index=1, channel_index=2):
    h, w = x.shape[row_index], x.shape[col_index]
    assert (h > hrg) and (w > wrg), "The size of cropping should smaller than the original image"
    if is_random:
        h_offset = int(np.random.uniform(0, h-hrg) - 1)
        w_offset = int(np.random.uniform(0, w-wrg) - 1)
        return x[h_offset: hrg + h_offset ,w_offset: wrg + w_offset]
    else:
        h_offset = int(np.floor((h - hrg)/ 2.))
        w_offset = int(np.floor((w - wrg)/ 2.))
        h_end = h_offset + hrg
        w_end = w_offset + wrg
        return x[h_offset: h_end, w_offset: w_end]

def flip_axis(x, axis, is_random=False):
    if is_random:
        factor = np.random.uniform(-1, 1)
        if factor > 0:
            x = np.asarray(x).swapaxes(axis, 0)
            x = x[::-1, ...]
            x = x.swapaxes(0, axis)
            return x
        else:
            return x
    else:
        x = np.asarray(x).swapaxes(axis, 0)
        x = x[::-1, ...]
        x = x.swapaxes(0, axis)
        return x

def imresize(x, size=[100, 100], interp='bilinear', mode=None):
    if x.shape[-1] == 1:
        x = scipy.misc.imresize(x[:, :, 0], size, interp=interp, mode=mode)
        return x[:, :, np.newaxis]
    elif x.shape[-1] == 3:
        return scipy.misc.imresize(x, size, interp=interp, mode=mode)
    else:
        raise Exception("Unsupported channel %d" % x.shape[-1])

def prepro_img(x, mode=None):
    if mode=='train':
        x = flip_axis(x, axis=1, is_random=True)
        x = rotation(x, rg=16, is_random=True, fill_mode='nearest')
        x = imresize(x, size=[256 + 60, 256 + 60], interp='bilinear', mode=None)
        x = crop(x, wrg=256, hrg=256, is_random=True)
        x = x / (255. / 2.)
        x = x - 1.
    return x

def save(saver, sess, logdir, step):
    model_name = 'model.ckpt'
    checkpoint_path = os.path.join(logdir, model_name)
    if not os.path.exists(logdir):
        os.makedirs(logdir)
    saver.save(sess, checkpoint_path, global_step=step)
    print('The checkpoint has been created.')

def load(saver, sess, ckpt_path):
    saver.restore(sess, ckpt_path)
    print("Restored model parameters from {}".format(ckpt_path))

In [19]:
import tensorflow.contrib.slim as slim

def resBlock(x, channels=64, kernel_size=[3,3], scale=1):
    tmp = slim.conv2d(x, channels, kernel_size, activation_fn=None)
    tmp = tf.nn.relu(tmp)
    tmp = slim.conv2d(tmp, channels, kernel_size, activation_fn=None)
    tmp *= scale
    return x + tmp

def upsample(x, scale=2, features=64, activation=tf.nn.relu):
    assert scale in [2,3,4]
    x = slim.conv2d(x, features, [3,3], activation_fn=activation)
    if scale == 2:
        ps_features = 3*(scale**2)
        x = slim.conv2d(x, ps_features, [3,3], activation_fn=activation)
        x = PS(x, 2, color=True)
    elif scale == 3:
        ps_features =3*(scale**2)
        x = slim.conv2d(x, ps_features, [3,3], activation_fn=activation)
        x = PS(x, 3, color=True)
    elif scale == 4:
        ps_features = 3*(2**2)
        for i in range(2):
            x = slim.conv2d(x, ps_features, [3,3], activation_fn=activation)
            x = PS(x, 2, color=True)
    return x

def _phase_shift(I, r):
    bsize, a, b, c = I.get_shape().as_list()
    bsize = tf.shape(I)[0]
    X = tf.reshape(I, (bsize, a, b, r, r))
    X = tf.transpose(X, (0, 1, 2, 4, 3))
    X = tf.split(X, a, 1)
    X = tf.concat([tf.squeeze(x, axis=1) for x in X],2)
    X = tf.split(X, b, 1)
    X = tf.concat([tf.squeeze(x, axis=1) for x in X],2)
    return tf.reshape(X, (bsize, a*r, b*r, 1))

def PS(X, r, color=False):
    if color:
        Xc = tf.split(X, 3, 3)
        X = tf.concat([_phase_shift(x, r) for x in Xc],3)
    else:
        X = _phase_shift(X, r)
    return X

def log10(x):
    numerator = tf.log(x)
    denominator = tf.log(tf.constant(10, dtype=numerator.dtype))
    return numerator / denominator

In [20]:
class EDSR(object):
    def __init__(self, img_size=64, num_layers=32, feature_size=256, scale=4, output_channels=3, lr=1e-4):
        print("Building EDSR...")
        """ Information """
        self.img_size = img_size
        self.scale = scale
        self.output_channels = output_channels
        self.lr = tf.Variable(lr, trainable=False)
        
        """ Place Holders"""
        self.input = x = tf.placeholder(tf.float32, [None, img_size, img_size, output_channels])
        self.target = y = tf.placeholder(tf.float32,[None, img_size*scale, img_size*scale, output_channels])
        image_input = x
        image_target = y
        
        """ Graph """
        x = slim.conv2d(image_input, feature_size, [3,3])
        conv_1 = x
        scaling_factor = 0.1
        for i in range(num_layers):
            x = resBlock(x, feature_size, scale=scaling_factor)
        x = slim.conv2d(x, feature_size, [3,3])
        x += conv_1
        x = upsample(x, scale, feature_size, None)
        self.out = output = tf.nn.tanh(x)
        
        """ Loss and Optimizer """
        self.loss = loss = tf.reduce_mean(tf.losses.absolute_difference(image_target, output))
        optimizer = tf.train.AdamOptimizer(self.lr)
        self.train_op = optimizer.minimize(self.loss)
        print("Building Complete!")

In [21]:
n_epochs = 100
batch_size = 16
lr = 1e-4
decay_every = 300
lr_decay = 0.5
train_samples_dir = 'EDSR_samples'
if os.path.exists(train_samples_dir) == False:
    os.makedirs(train_samples_dir)
checkpoint_dir = './EDSR'

In [22]:
tf.reset_default_graph()
model = EDSR()

Building EDSR...
Building Complete!


In [23]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
sess = tf.Session(config=config)
init = tf.global_variables_initializer()
sess.run(init)
saver = tf.train.Saver(var_list=tf.global_variables(), max_to_keep=10)
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
if ckpt and ckpt.model_checkpoint_path:
    loader = tf.train.Saver(var_list=tf.global_variables())
    load_step = int(os.path.basename(ckpt.model_checkpoint_path).split('-')[1])
    load(loader, sess, ckpt.model_checkpoint_path)
else:
    print('no checkpoints find.')

print('[*] Training Start! [*]')
for epoch in range(n_epochs):
    start_time = time.time()
    if epoch !=0 and (epoch % decay_every == 0):
        new_lr_decay = lr_decay ** (epoch // decay_every)
        sess.run(tf.assign(model.lr, lr * new_lr_decay))
        log = " ** new learning rate: %f" % (lr * new_lr_decay)
        print(log)
    elif epoch == 0:
        log = " ** init lr: %f, decay_every_epoch: %d, lr_decay: %f" % (lr, decay_every, lr_decay)
        print(log)     
    n_batches = len(train_images) // batch_size
    random_list = np.random.permutation(len(train_images))
    for iteration in range(n_batches):
        step_time = time.time()
        idx = random_list[iteration*batch_size:(iteration+1)*batch_size]
        y = train_images[idx]
        y = threading_data(y, prepro_img, mode='train')
        x = []
        for image in y:
            x.append(resize(image, (64, 64, 3)))
        x = np.asarray(x)
        assert x.shape[0] == y.shape[0]
        _, loss = sess.run([model.train_op, model.loss], feed_dict={model.input: x, model.target: y})
        if (iteration + 1) % (n_batches // 10) == 0:
            print('Iteration: [%3d/%3d] time: %4.4fs, loss: %.8f' % (iteration, n_batches, time.time() - step_time, loss))
    if (epoch + 1) % 1 == 0:
        print(" ** Epoch %d took %fs" % (epoch, time.time()-start_time))
        random_list = np.random.permutation(len(train_images))
        idx = random_list[0: batch_size]
        sample_images_64 = train_images_64[idx]
        sample_outputs = sess.run([model.out], feed_dict={model.input: sample_images_64})
        sample_outputs = np.reshape(sample_outputs, [batch_size, 256, 256, 3])
        save_images(sample_outputs, [4, 4], 'EDSR_samples/train_{:02d}.png'.format(epoch))  
    if (epoch != 0) and (epoch % 2) == 0:
        save(saver, sess, checkpoint_dir, epoch)
        print("[*] Save checkpoints SUCCESS!")
print('[*] Training Complete! [*]')

no checkpoints find.
[*] Training Start! [*]
 ** init lr: 0.000100, decay_every_epoch: 300, lr_decay: 0.500000


/home/chszerg/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/ipykernel_launcher.py:142: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


Iteration: [ 54/553] time: 1.3056s, loss: 0.28635725
Iteration: [109/553] time: 1.3176s, loss: 0.24112408
Iteration: [164/553] time: 1.3508s, loss: 0.20293885
Iteration: [219/553] time: 1.3564s, loss: 0.19479351
Iteration: [274/553] time: 1.3553s, loss: 0.14360212
Iteration: [329/553] time: 1.3728s, loss: 0.11974975
Iteration: [384/553] time: 1.4228s, loss: 0.10955516
Iteration: [439/553] time: 1.3568s, loss: 0.11014163
Iteration: [494/553] time: 1.3677s, loss: 0.10967568
Iteration: [549/553] time: 1.3953s, loss: 0.10548107
 ** Epoch 0 took 763.553990s


/home/chszerg/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Iteration: [ 54/553] time: 1.3770s, loss: 0.10154118
Iteration: [109/553] time: 1.3111s, loss: 0.09531797
Iteration: [164/553] time: 1.3496s, loss: 0.10053264
Iteration: [219/553] time: 1.4026s, loss: 0.09917217
Iteration: [274/553] time: 1.4059s, loss: 0.09351692
Iteration: [329/553] time: 1.3879s, loss: 0.08917459
Iteration: [384/553] time: 1.3350s, loss: 0.07942794
Iteration: [439/553] time: 1.3979s, loss: 0.07511904
Iteration: [494/553] time: 1.3659s, loss: 0.08743627
Iteration: [549/553] time: 1.3440s, loss: 0.08111409
 ** Epoch 1 took 755.512177s
Iteration: [ 54/553] time: 1.3769s, loss: 0.06736470
Iteration: [109/553] time: 1.3823s, loss: 0.07507487
Iteration: [164/553] time: 1.3763s, loss: 0.06960527
Iteration: [219/553] time: 1.3478s, loss: 0.06199709
Iteration: [274/553] time: 1.3752s, loss: 0.06547598
Iteration: [329/553] time: 1.4421s, loss: 0.07337110
Iteration: [384/553] time: 1.3294s, loss: 0.08007286
Iteration: [439/553] time: 1.3907s, loss: 0.07113817
Iteration: [494/5

...

Iteration: [494/553] time: 1.3378s, loss: 0.03441408
Iteration: [549/553] time: 1.4332s, loss: 0.03312508
 ** Epoch 42 took 762.630690s
The checkpoint has been created.
[*] Save checkpoints SUCCESS!
Iteration: [ 54/553] time: 1.3778s, loss: 0.03084271
Iteration: [109/553] time: 1.4010s, loss: 0.02555290
Iteration: [164/553] time: 1.4094s, loss: 0.02631533
Iteration: [219/553] time: 1.4060s, loss: 0.02731978
Iteration: [274/553] time: 1.3774s, loss: 0.03083277
Iteration: [329/553] time: 1.3733s, loss: 0.04141834
Iteration: [384/553] time: 1.3482s, loss: 0.02554601
Iteration: [439/553] time: 1.3588s, loss: 0.02702292
Iteration: [494/553] time: 1.4000s, loss: 0.03598133
Iteration: [549/553] time: 1.3521s, loss: 0.03519825
 ** Epoch 43 took 762.636911s
Iteration: [ 54/553] time: 1.4237s, loss: 0.03415799
Iteration: [109/553] time: 1.3885s, loss: 0.03133393
Iteration: [164/553] time: 1.3412s, loss: 0.03234592
Iteration: [219/553] time: 1.3879s, loss: 0.02430725
Iteration: [274/553] time: 1.

Iteration: [439/553] time: 1.3723s, loss: 0.03056932
Iteration: [494/553] time: 1.3658s, loss: 0.03618547
Iteration: [549/553] time: 1.3110s, loss: 0.02870081
 ** Epoch 56 took 759.138414s
The checkpoint has been created.
[*] Save checkpoints SUCCESS!
Iteration: [ 54/553] time: 1.3906s, loss: 0.03636632
Iteration: [109/553] time: 1.3589s, loss: 0.03650638
Iteration: [164/553] time: 1.4027s, loss: 0.03188013
Iteration: [219/553] time: 1.3674s, loss: 0.04489919
Iteration: [274/553] time: 1.3654s, loss: 0.02839536
Iteration: [329/553] time: 1.3817s, loss: 0.03157411
Iteration: [384/553] time: 1.3266s, loss: 0.03235429
Iteration: [439/553] time: 1.3759s, loss: 0.04170618
Iteration: [494/553] time: 1.4028s, loss: 0.03275072
Iteration: [549/553] time: 1.3433s, loss: 0.03649639
 ** Epoch 57 took 759.868933s
Iteration: [ 54/553] time: 1.4263s, loss: 0.03321511
Iteration: [109/553] time: 1.4087s, loss: 0.03117308
Iteration: [164/553] time: 1.3553s, loss: 0.02672257
Iteration: [219/553] time: 1.

Iteration: [384/553] time: 1.3914s, loss: 0.03249271
Iteration: [439/553] time: 1.4135s, loss: 0.03302428
Iteration: [494/553] time: 1.4396s, loss: 0.03482375
Iteration: [549/553] time: 1.3907s, loss: 0.02767854
 ** Epoch 70 took 763.440668s
The checkpoint has been created.
[*] Save checkpoints SUCCESS!
Iteration: [ 54/553] time: 1.3560s, loss: 0.03013272
Iteration: [109/553] time: 1.4296s, loss: 0.03496234
Iteration: [164/553] time: 1.3190s, loss: 0.03033215
Iteration: [219/553] time: 1.3572s, loss: 0.03161832
Iteration: [274/553] time: 1.3460s, loss: 0.03105221
Iteration: [329/553] time: 1.3690s, loss: 0.02869163
Iteration: [384/553] time: 1.3371s, loss: 0.02799138
Iteration: [439/553] time: 1.3335s, loss: 0.03364244
Iteration: [494/553] time: 1.4019s, loss: 0.02831955
Iteration: [549/553] time: 1.4094s, loss: 0.02577720
 ** Epoch 71 took 762.781962s
Iteration: [ 54/553] time: 1.3508s, loss: 0.03249309
Iteration: [109/553] time: 1.3876s, loss: 0.02616436
Iteration: [164/553] time: 1.

Iteration: [329/553] time: 1.3902s, loss: 0.02424816
Iteration: [384/553] time: 1.3345s, loss: 0.03750855
Iteration: [439/553] time: 1.3189s, loss: 0.02893978
Iteration: [494/553] time: 1.3849s, loss: 0.02839720
Iteration: [549/553] time: 1.3492s, loss: 0.03027426
 ** Epoch 84 took 762.547923s
The checkpoint has been created.
[*] Save checkpoints SUCCESS!
Iteration: [ 54/553] time: 1.3772s, loss: 0.03001765
Iteration: [109/553] time: 1.3043s, loss: 0.03582750
Iteration: [164/553] time: 1.3845s, loss: 0.02692208
Iteration: [219/553] time: 1.3756s, loss: 0.03087166
Iteration: [274/553] time: 1.4029s, loss: 0.03311759
Iteration: [329/553] time: 1.3517s, loss: 0.02516099
Iteration: [384/553] time: 1.3441s, loss: 0.03214614
Iteration: [439/553] time: 1.3531s, loss: 0.02767807
Iteration: [494/553] time: 1.4278s, loss: 0.03142519
Iteration: [549/553] time: 1.3253s, loss: 0.03243485
 ** Epoch 85 took 763.517150s
Iteration: [ 54/553] time: 1.3727s, loss: 0.03159254
Iteration: [109/553] time: 1.

Iteration: [274/553] time: 1.3694s, loss: 0.02660182
Iteration: [329/553] time: 1.3093s, loss: 0.03065497
Iteration: [384/553] time: 1.4052s, loss: 0.03559192
Iteration: [439/553] time: 1.3550s, loss: 0.02824754
Iteration: [494/553] time: 1.2926s, loss: 0.03133195
Iteration: [549/553] time: 1.3486s, loss: 0.03594369
 ** Epoch 98 took 750.372663s
The checkpoint has been created.
[*] Save checkpoints SUCCESS!
Iteration: [ 54/553] time: 1.4497s, loss: 0.02778061
Iteration: [109/553] time: 1.3572s, loss: 0.03176140
Iteration: [164/553] time: 1.3532s, loss: 0.02617385
Iteration: [219/553] time: 1.3981s, loss: 0.03992768
Iteration: [274/553] time: 1.3372s, loss: 0.02277428
Iteration: [329/553] time: 1.3246s, loss: 0.03194448
Iteration: [384/553] time: 1.3827s, loss: 0.03681608
Iteration: [439/553] time: 1.3816s, loss: 0.03045992
Iteration: [494/553] time: 1.3874s, loss: 0.02650729
Iteration: [549/553] time: 1.3588s, loss: 0.03680688
 ** Epoch 99 took 752.585604s
[*] Training Complete! [*]


In [ ]:
def get_model_params(sess):
    gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {gvar.op.name: value for gvar, value in zip(gvars, sess.run(gvars))}

In [ ]:
all_vars = [var for var in tf.trainable_variables()]

In [ ]:
rnn_vars = [var for var in tf.trainable_variables() if 'rnnencoder' in var.name]